In [ ]:
import math
import numpy as np
import dask

from scipy.stats import gaussian_kde

from common import *
from mcs_shared import load_als_depth, load_isnobal_depth, load_factors_tif, \
    ACCUMULATION_FLIGHTS
from snobedo.lib.dask_utils import run_with_client

In [ ]:
%load_ext autoreload
%autoreload 2

## Notes

* Resolution: 100m
* ALS depth filtered to 0.1m and 5m
    * Depths below 0.1m had factors in the thousands
* Each day is reduced to only pixels with measured ALS depths

## Snow Depth Map resolution

In [ ]:
RESOLUTION = 10 # meters

COLORS = [
    'royalblue', 'peru', 'chartreuse', 'lightcoral', 'blueviolet', 'seagreen'
]
COLOR_MAP = { flight: COLORS[idx] for idx, flight in enumerate(ACCUMULATION_FLIGHTS) }

WORKERS=dict(cores=6, memory=30)

## Get common area across all flights

In [ ]:
pd.options.plotting.backend = 'matplotlib'

In [ ]:
@dask.delayed
def kde_density(data):
    kde = gaussian_kde(data)
    x = np.linspace(0, 5, 500)
    density = kde(x)
    return x, density

def plot_densities(density_data):
    fig, ax = plt.subplots(dpi=300, figsize=(10, 4))
    
    for flight, data in density_data.items():
        ax.plot(
            data[0], data[1], 
            label=pd.to_datetime(flight).strftime('%Y-%m-%d'),
            lw=1, color=COLOR_MAP[flight], alpha=0.8
        )
    return ax

In [ ]:
def load_data(flights):
    als_depth = {}
    isnobal_depth = {}
    factor_maps = {}

    for flight in flights:
        als = load_als_depth(flight, RESOLUTION)
        isnobal = load_isnobal_depth(flight, RESOLUTION)
        factors = load_factors_tif(flight, RESOLUTION)

        # Mask to common area
        mask = np.isnan(factors)
        als = als[~mask]
        isnobal = isnobal[~mask]
        factors = factors[~mask]
        
        date = pd.to_datetime(flight).strftime('%Y-%m-%d')

        isnobal_depth[flight] = pd.Series(isnobal.flatten(), name=date)
        als_depth[flight] = pd.Series(als.flatten(), name=date)
        factor_maps[flight] = pd.Series(factors.flatten(), name=date)

    return als_depth, isnobal_depth, factor_maps

In [ ]:
als_depths, isnobal_depths, factors = load_data(ACCUMULATION_FLIGHTS)

## ALS Depths

In [ ]:
with run_with_client(**WORKERS):
    densities = dask.compute({
        date: kde_density(ds.values) for date, ds in als_depths.items()
    })[0]

In [ ]:
ax = plot_densities(densities)
ax.set_xlim(0, 4.5)
ax.set_xlabel('Snow Depth (m)')
ax.set_ylim(-0.05, 2)
ax.set_ylabel('Densities')
ax.set_title("ALS flights")
ax.legend();

In [ ]:
df_als = pd.concat(als_depths, axis=1)
df_als.describe(percentiles=[0.01, 0.5, 0.99])

### Normalized $\frac{depth_i}{mean_{depth}}$

In [ ]:
with run_with_client(**WORKERS):
    densities = dask.compute({
        date: kde_density(ds.values/ds.mean()) for date, ds in als_depths.items()
    })[0]

In [ ]:
ax = plot_densities(densities)
ax.set_xlim(0,2)
ax.set_xlabel('ALS - Normalized Depth')
ax.set_ylim(-0.05, 2)
ax.set_ylabel('Densities')
ax.set_title("ALS flights")
ax.legend();

## Model Depths

In [ ]:
with run_with_client(**WORKERS):
    densities = dask.compute({
        date: kde_density(ds.values) for date, ds in isnobal_depths.items()
    })[0]

In [ ]:
ax = plot_densities(densities)
ax.set_xlim(0, 4.5)
ax.set_xlabel('Snow Depth (m)')
ax.set_ylim(-0.05, 3.5)
ax.set_ylabel('Densities')
ax.set_title("iSnobal Depths")
ax.legend();

### Normalized $\frac{depth_i}{mean_{depth}}$

In [ ]:
with run_with_client(**WORKERS):
    densities = dask.compute({
        date: kde_density(ds.values/ds.mean()) for date, ds in isnobal_depths.items()
    })[0]

In [ ]:
ax = plot_densities(densities)
ax.set_xlim(0,2)
ax.set_xlabel('iSnobal - Normalized Depth')
ax.set_ylim(-0.05, 5.5)
ax.set_ylabel('Densities')
ax.set_title("iSnobal Depths")
ax.legend();

In [ ]:
df_isnobal = pd.concat(isnobal_depths, axis=1)

In [ ]:
df_isnobal.describe(percentiles=[0.01, 0.5, 0.99])

## Factors

In [ ]:
with run_with_client(**WORKERS):
    densities = dask.compute({
        date: kde_density(ds.values) for date, ds in factors.items()
    })[0]

In [ ]:
ax = plot_densities(densities)
ax.set_xlim(0.25, 3)
ax.set_xlabel('Factors')
ax.set_ylim(-0.05, 3.5)
ax.set_ylabel('Densities')
ax.set_title("Factors")
ax.legend();

In [ ]:
df_factors = pd.concat(factors, axis=1)
df_factors.describe(percentiles=[0.01, 0.5, 0.99])

## Holoviews

In [ ]:
from holoviews import opts, Cycle

In [ ]:
use_hvplot()

### Density plots 

In [ ]:
def plot_flights(flights, normalized_depths, als_depth, title):
    return hv.Layout([
            hv.NdOverlay(
                { flights[n]: hv.Distribution(normalized_depths[n]) for n in range(0, len(flights)) }
            ).opts(
                opts.Distribution(filled=False, line_color=Cycle())
            ).opts(**HV_PLOT_OPTS, title='Depths', xlim=(0, 2), xlabel='Normalized Depth'),
            hv.NdOverlay(
                { 
                    flights[n]: hv.Distribution(factors[n]) for n in range(0, len(flights)) 
                }
            ).opts(
                opts.Distribution(filled=False, line_color=Cycle())
            ).opts(**HV_PLOT_OPTS, title='Factors', xlabel='Factors'),
            hv.NdOverlay(
                { 
                    flights[n]: hv.Distribution(als_depths[n])
                    for n in range(0, len(flights)) 
                }
            ).opts(
                opts.Distribution(filled=False, line_color=Cycle())
            ).opts(**HV_PLOT_OPTS, title='Depths', xlim=(0, 4), xlabel='Depths')
        ]).opts(shared_axes=False, title=title).cols(1)

In [ ]:
plot_flights(ACCUMULATION_FLIGHTS, normalized_depths, als_depth, "Accumulation Flights")

## Gamma

Plot CDF for depths 

In [ ]:
from scipy import stats
import pprint

In [ ]:
def fit_norm(flight, idx, x):
    param = stats.norm.fit(flight)
    return stats.norm.cdf(x, *param)

In [ ]:
def fit_gamma(flight, idx, x):
    param = stats.gamma.fit(flight)
    return stats.gamma.cdf(x, *param)    

In [ ]:
fig, ax = plt.subplots(dpi=300)

for idx, flight in enumerate(factors):
    ecdf_f = stats.ecdf(flight)
    ecdf_f.cdf.plot(ax, label=f'{ACCUMULATION_FLIGHTS[idx]}')
    # print(peak_als_depth[idx])
    # pprint.pp(ecdf_f.cdf.quantiles)

ax.set_title("ECDF")
ax.legend()
plt.show()